In [6]:
# https://www.analyticsvidhya.com/datahack/contest/wns-analytics-hackathon-2018-1/

In [8]:
!pip install imblearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.4/238.4 kB 1.0 MB/s eta 0:00:002 kB/s eta 0:00:0101


### Importing Libraries

In [53]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import KFold

### Reading data

In [2]:
file_path = '~/Desktop_linux/chinu/HR_analytics_vidya/train_LZdllcl.csv'
df = pd.read_csv(file_path)
df.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
0,65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,1,0,49,0
1,65141,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,0,60,0
2,7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,0,50,0
3,2542,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,0,50,0
4,48945,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,0,73,0


In [3]:
file_path = '/home/pc/Desktop_linux/chinu/HR_analytics_vidya/test_2umaH9m.csv'
unseen_df = pd.read_csv(file_path)
unseen_df.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score
0,8724,Technology,region_26,Bachelor's,m,sourcing,1,24,NaN,1,1,0,77
1,74430,HR,region_4,Bachelor's,f,other,1,31,3.0,5,0,0,51
2,72255,Sales & Marketing,region_13,Bachelor's,m,other,1,31,1.0,4,0,0,47
3,38562,Procurement,region_2,Bachelor's,f,other,3,31,2.0,9,0,0,65
4,64486,Finance,region_29,Bachelor's,m,sourcing,1,30,4.0,7,0,0,61


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54808 entries, 0 to 54807
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   employee_id           54808 non-null  int64  
 1   department            54808 non-null  object 
 2   region                54808 non-null  object 
 3   education             52399 non-null  object 
 4   gender                54808 non-null  object 
 5   recruitment_channel   54808 non-null  object 
 6   no_of_trainings       54808 non-null  int64  
 7   age                   54808 non-null  int64  
 8   previous_year_rating  50684 non-null  float64
 9   length_of_service     54808 non-null  int64  
 10  KPIs_met >80%         54808 non-null  int64  
 11  awards_won?           54808 non-null  int64  
 12  avg_training_score    54808 non-null  int64  
 13  is_promoted           54808 non-null  int64  
dtypes: float64(1), int64(8), object(5)
memory usage: 5.9+ MB


In [5]:
unseen_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23490 entries, 0 to 23489
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   employee_id           23490 non-null  int64  
 1   department            23490 non-null  object 
 2   region                23490 non-null  object 
 3   education             22456 non-null  object 
 4   gender                23490 non-null  object 
 5   recruitment_channel   23490 non-null  object 
 6   no_of_trainings       23490 non-null  int64  
 7   age                   23490 non-null  int64  
 8   previous_year_rating  21678 non-null  float64
 9   length_of_service     23490 non-null  int64  
 10  KPIs_met >80%         23490 non-null  int64  
 11  awards_won?           23490 non-null  int64  
 12  avg_training_score    23490 non-null  int64  
dtypes: float64(1), int64(7), object(5)
memory usage: 2.3+ MB


### Understanding data

In [21]:
for feature in df.columns:
    print(df[feature].nunique())
    print(df[feature].value_counts())
    print("*" * 40)


54808
employee_id
51526    1
65438    1
65141    1
7513     1
2542     1
        ..
60051    1
49017    1
29934    1
28911    1
73202    1
Name: count, Length: 54808, dtype: int64
****************************************
9
department
Sales & Marketing    16840
Operations           11348
Technology            7138
Procurement           7138
Analytics             5352
Finance               2536
HR                    2418
Legal                 1039
R&D                    999
Name: count, dtype: int64
****************************************
34
region
region_2     12343
region_22     6428
region_7      4843
region_15     2808
region_13     2648
region_26     2260
region_31     1935
region_4      1703
region_27     1659
region_16     1465
region_28     1318
region_11     1315
region_23     1175
region_29      994
region_32      945
region_19      874
region_20      850
region_14      827
region_25      819
region_17      796
region_5       766
region_6       690
region_30      657
region_8 

### Missing data

In [36]:
df.isna().sum()

employee_id                0
department                 0
region                     0
education                  0
gender                     0
recruitment_channel        0
no_of_trainings            0
age                        0
previous_year_rating    4124
length_of_service          0
KPIs_met >80%              0
awards_won?                0
avg_training_score         0
is_promoted                0
dtype: int64

In [33]:
unseen_df.isna().sum()

employee_id                0
department                 0
region                     0
education               1034
gender                     0
recruitment_channel        0
no_of_trainings            0
age                        0
previous_year_rating    1812
length_of_service          0
KPIs_met >80%              0
awards_won?                0
avg_training_score         0
dtype: int64

In [49]:
# Education column
df['education'] = df['education'].fillna(df['education'].mode()[0])
unseen_df['education'] = unseen_df['education'].fillna(df['education'].mode()[0])

# previous_year_rating column
rating_fill_values = {0: df.loc[df['is_promoted'] == 0, 'previous_year_rating'].mode()[0],
                      1: df.loc[df['is_promoted'] == 1, 'previous_year_rating'].mode()[0]}

df.loc[df['is_promoted'] == 0, 'previous_year_rating'] = df.loc[df['is_promoted'] == 0, 'previous_year_rating'].fillna(rating_fill_values[0])
df.loc[df['is_promoted'] == 1, 'previous_year_rating'] = df.loc[df['is_promoted'] == 1, 'previous_year_rating'].fillna(rating_fill_values[1])
# assuming that most of the data is for class 0, filling the value of mode for class 0 
unseen_df['previous_year_rating'] = unseen_df['previous_year_rating'].fillna(rating_fill_values[0])

### Target Encoding for 'region' feature

In [ ]:
# Initialize new column
df['region_encoded'] = 0
unseen_df['region_encoded'] = 0

kf = KFold(n_splits=5, shuffle=True, random_state=42)
for train_idx, test_idx in kf.split(df):
    X_train, X_test = df.iloc[train_idx], df.iloc[test_idx]

    # calculate mean target for each region based on train data only
    region_mean = X_train.groupby('region')['is_promoted'].mean()

    df.loc[df.index[test_idx], 'region_encoded'] = df['region'][test_idx].map(region_mean)

region
region_1     0.104701
region_10    0.073864
region_11    0.053221
region_12    0.071979
region_13    0.091647
region_14    0.072643
region_15    0.080808
region_16    0.073005
region_17    0.143084
region_18    0.000000
region_19    0.059238
region_2     0.080088
region_20    0.065185
region_21    0.039157
region_22    0.114732
region_23    0.116329
region_24    0.037129
region_25    0.120370
region_26    0.063616
region_27    0.076923
region_28    0.126747
region_29    0.042308
region_3     0.114391
region_30    0.092453
region_31    0.057180
region_32    0.041835
region_33    0.036199
region_34    0.025862
region_4     0.139398
region_5     0.050489
region_6     0.047101
region_7     0.105318
region_8     0.083821
region_9     0.020290
Name: is_promoted, dtype: float64
region
region_1     0.077406
region_10    0.072581
region_11    0.060725
region_12    0.074257
region_13    0.087959
region_14    0.079460
region_15    0.080795
region_16    0.073024
region_17    0.138462
region

## SMOTE - Over Sampling for imbalanced data

In [10]:
smote = SMOTE(sampling_strategy=1.0, random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

NameError: name 'X_train' is not defined

### Feature Scaling

In [ ]:
# Step 7: Feature Scaling (for models like Logistic Regression)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 8: Train Logistic Regression with Class Weights
log_reg = LogisticRegression(class_weight='balanced', random_state=42)
log_reg.fit(X_train_scaled, y_train)

# Step 9: Train Random Forest with Class Weights
rf_clf = RandomForestClassifier(class_weight='balanced', n_estimators=100, random_state=42)
rf_clf.fit(X_train, y_train)